  
<td>
    <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

----

# Model Diagnostics


Throughout the process of training your machine learning (ML) model, you may want to investigate your model's failures in order to understand which areas need improvement. Looking at an error analysis after each training iteration can help you understand whether you need to revise your annotations, make your ontology more clear, or create more training data that targets a specific area.
Labelbox now offers a Model Diagnostics tool that analyzes the performance of your model's predictions in a single interface.
With Model Diagnostics, you can:
*   Inspect model behavior across experiments
*   Adjust model hyperparameters and visualize model failures
*   Use the Python SDK to create the analysis pipeline

## How it works

Configuring Model Diagnostics is all done via the SDK. We have created a Google colab notebook to demonstrate this process. The notebook also includes a section that leverages MAL in order to quickly create ground truth annotations.
An Experiment is a specific instance of a model generating output in the form of predictions.
In Labelbox, the `Model` object represents your ML model and it is what you'll be performing experiments on. It references a set of annotations specified by an ontology. 
The `Model Run` object represents the experiment itself. It is a specific instance of a `Model` with preconfigured hyperparameters (training data). You can upload inferences across each `Model Run`, filter by IoU score, and compare your model's predictions against the annotations from your training data.

## Steps
1. Make sure you are signed up for the beta. If not navigate here https://labelbox.com/product/model-diagnostics
2. Have a set of ground truth labels in a project
3. Install the latest SDK release (At this time that is 3.0.0rc1)
4. Create a `Model`
5. Create a `Model Run`
6. Compute predictions
7. Compute model performance metrics
8. Upload labels, predictions, and metrics
9. Navigate to the `Models` tab on Labelbox

## Best practices
Currently there is a limit of 2000 images per model run. We suggest uploading lower performing examples from your test set.


## Environment Setup

Install dependencies

In [ ]:
!pip install -q "labelbox[data]" \
             scikit-image \
             tensorflow

In [2]:
# Run these if running in a colab notebook
COLAB = "google.colab" in str(get_ipython())

if COLAB:
    !git clone https://github.com/Labelbox/labelbox-python.git
    !cd labelbox-python
    !mv labelbox-python/examples/model_assisted_labeling/*.py .

Import libraries

In [1]:
import sys
sys.path.append('../model_assisted_labeling')

import uuid
import numpy as np
from skimage import measure
import requests
from tqdm import notebook
import requests
import csv
import os

from labelbox.schema.ontology import OntologyBuilder, Tool
from labelbox.data.metrics.group import get_label_pairs
from labelbox.data.metrics import feature_miou_metric, feature_confusion_matrix_metric
from labelbox import Client, LabelingFrontend, MALPredictionImport
from labelbox.data.serialization import NDJsonConverter
from labelbox.data.annotation_types import (
    ScalarMetric, 
    LabelList, 
    Label, 
    ImageData, 
    MaskData,
    Mask, 
    Polygon,
    Point, 
    Rectangle, 
    ObjectAnnotation
)

try:
    from image_model import predict, load_model
except ModuleNotFoundError: 
    # !git clone https://github.com/Labelbox/labelbox-python.git
    # !cd labelbox-python && git checkout mea-dev
    # !mv labelbox-python/examples/model_assisted_labeling/*.py .
    raise Exception("You will need to run from the labelbox-python git repo")

2021-09-13 09:27:18.654668: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-13 09:27:18.655024: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Configure client

In [3]:
API_KEY = None
PROJECT_NAME = "Diagnostics Demo"
MODEL_NAME = "MSCOCO-Mapillary"
MODEL_VERSION = "0.0.0"

In [4]:
client = Client(api_key=API_KEY)
load_model() # initialize Tensorflow Model

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from gs://cloud-tpu-checkpoints/mask-rcnn/1555659850/variables/variables


2021-09-13 09:27:38.928229: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:196] None of the MLIR optimization passes are enabled (registered 0 passes)


In [5]:
# Configure for whatever combination of tools and class names that you would like.
class_mappings = {
    1: {"name": 'person', "kind": Tool.Type.POLYGON},
    2: {"name": 'bicycle', "kind": Tool.Type.SEGMENTATION, 'color' : 64},
    3: {"name": 'car', "kind": Tool.Type.BBOX},
    4: {"name": 'motorcycle', "kind": Tool.Type.BBOX},
    6: {"name": 'bus', "kind": Tool.Type.POLYGON},
    7: {"name": 'train', "kind": Tool.Type.POLYGON},
    8: {"name": 'truck', "kind": Tool.Type.POLYGON},
    10: {"name": 'traffic light', "kind": Tool.Type.POINT},
    11: {"name": 'fire hydrant', "kind": Tool.Type.BBOX},
    13: {"name": 'stop sign', "kind": Tool.Type.SEGMENTATION, 'color' : 255},
    14: {"name": 'parking meter', "kind": Tool.Type.POINT},
    28: {"name": 'umbrella', "kind": Tool.Type.SEGMENTATION, 'color' : 128},    
    31: {"name": 'handbag', "kind": Tool.Type.POINT},        
}

## Create Predictions
* Loop over data_rows, make predictions, and create ndjson

In [6]:
# --- setup dataset ---
# load mapillary sample
sample_csv_url = "https://raw.githubusercontent.com/Labelbox/labelbox-python/develop/examples/assets/mapillary_sample.csv"
with requests.get(sample_csv_url, stream=True) as r:
    image_data = [row.split(',') for row in (line.decode('utf-8') for line in r.iter_lines())]

In [7]:
predictions = LabelList()
for (image_url, external_id) in notebook.tqdm(image_data[:10]):
    image = ImageData(url = image_url, external_id = external_id)
    height, width = image.value.shape[:2]
    prediction = predict(np.array([image.im_bytes]), min_score=0.5, height=height, width = width)
    boxes, classes, seg_masks = prediction["boxes"], prediction["class_indices"], prediction["seg_masks"]
    annotations = []
    for box, class_idx, seg in zip(boxes, classes, seg_masks):
        if class_idx in class_mappings:
            class_info = class_mappings.get(class_idx)
            if class_info['kind'] == Tool.Type.POLYGON:
                contours = measure.find_contours(seg, 0.5)
                pts = contours[0].astype(np.int32)
                value = Polygon(points = [Point(x = x, y = y) for x,y in np.roll(pts, 1, axis=-1)])
            elif class_info['kind'] == Tool.Type.BBOX:
                value = Rectangle(start = Point(x = box[1], y = box[0]), end = Point(x=box[3], y=box[2]))
            elif class_info['kind'] == Tool.Type.POINT:
                value = Point(x=(box[1] + box[3]) / 2., y = (box[0] + box[2]) / 2.)
            elif class_info['kind'] == Tool.Type.SEGMENTATION:
                value = Mask(mask = MaskData.from_2D_arr(seg * class_info['color']), color = (class_info['color'],)* 3)
            else:
                raise ValueError(f"Unsupported kind found. {class_info['kind']}")
            annotations.append(ObjectAnnotation(name = class_info['name'], value = value))
    predictions.append(Label(data = image, annotations = annotations))

  0%|          | 0/10 [00:00<?, ?it/s]

## Setup a project

In [8]:
# --- Use the class mapping specified above ( Will include all specified classes )
tools = []
for target in class_mappings.values():
     tools.append(Tool(tool=target['kind'], name=target["name"]))
ontology_builder = OntologyBuilder(tools=tools)

# --- Optionally Setup ontology from predictions ( Only will include predicted classes )
#ontology_builder = predictions.get_ontology()

In [9]:
print(f"Setting up: {PROJECT_NAME}")

project = client.create_project(name=PROJECT_NAME)
editor = next(client.get_labeling_frontends(where=LabelingFrontend.name == "Editor"))
project.setup(editor, ontology_builder.asdict())

dataset = client.create_dataset(name="Mapillary Diagnostics Demo")
print(f"Dataset Created: {dataset.uid}")
project.datasets.connect(dataset)

Setting up: Diagnostics Demo
Dataset Created: cktiom65aclzm0y896zeua5ne


## Prepare for upload
* Our local annotations need the following:
    1. signed url for segmentation masks
    2. data rows in labelbox
    3. feature schema ids

In [10]:
signer = lambda _bytes: client.upload_data(content=_bytes, sign=True)
predictions.add_url_to_masks(signer) \
         .add_url_to_data(signer) \
         .assign_feature_schema_ids(OntologyBuilder.from_project(project)) \
         .add_to_dataset(dataset, client.upload_data)

10it [00:00, 63358.07it/s]
10it [00:00, 76818.75it/s]
10it [00:00, 60963.72it/s]


## **Optional** - Create labels with [Model Assisted Labeling](https://docs.labelbox.com/en/core-concepts/model-assisted-labeling)

* Pre-label image so that we can quickly create ground truth
* Create ground truth data for Model Diagnostics
* Click on link below to label

In [11]:
RUN_MAL = True
if RUN_MAL:
    project.enable_model_assisted_labeling()
    # Convert from annotation types to import format
    ndjson_predictions = NDJsonConverter.serialize(predictions)
    upload_task = MALPredictionImport.create_from_objects(client, project.uid, f'mal-import-{uuid.uuid4()}',ndjson_predictions )
    upload_task.wait_until_done()
    print(upload_task.state , '\n')

AnnotationImportState.FINISHED 



In [12]:
print(f"https://app.labelbox.com/go-label/{project.uid}")

https://app.labelbox.com/go-label/cktiom4y2czqg0y9u9qqfdvhk


## Export Labels

We do not support `Skipped` labels and have a limit of **2000**

In [13]:
MAX_LABELS = 2000
labels = [l for idx, l in enumerate(project.label_generator()) if idx < MAX_LABELS]

## Setup Model & Model Run

In [14]:
lb_model = client.create_model(name = MODEL_NAME, ontology_id = project.ontology().uid)
lb_model_run = lb_model.create_model_run(MODEL_VERSION)

Select label ids to upload

In [15]:
lb_model_run.upsert_labels([label.uid for label in labels])

True

### Compute Metrics

In [16]:
pairs = get_label_pairs(labels, predictions, filter_mismatch = True)
for (ground_truth, prediction) in pairs.values():
    metrics = []
    metrics.extend(feature_miou_metric(ground_truth.annotations, prediction.annotations))
    metrics.extend(feature_confusion_matrix_metric(ground_truth.annotations, prediction.annotations))    
    prediction.annotations.extend(metrics)

### Upload to Labelbox

In [17]:
upload_task = lb_model_run.add_predictions(f'diagnostics-import-{uuid.uuid4()}', NDJsonConverter.serialize(predictions))
upload_task.wait_until_done()
print(upload_task.state)

AnnotationImportState.FINISHED


### Open Model Run

In [18]:
for idx, annotation_group in enumerate(lb_model_run.annotation_groups()):
    if idx == 5:
        break
    print(annotation_group.url)

https://app.labelbox.com/models/9c4cba6e-efb4-03f1-302e-d5718c4eca20/9c4cba6f-8ea1-0202-ec4e-528291bf932e/AllDatarowsSlice/25da00a5-e451-4dbd-824d-a0425ce94ffa?view=carousel
https://app.labelbox.com/models/9c4cba6e-efb4-03f1-302e-d5718c4eca20/9c4cba6f-8ea1-0202-ec4e-528291bf932e/AllDatarowsSlice/3d20b8e6-5584-45fd-b0a4-021b3a16f25b?view=carousel
https://app.labelbox.com/models/9c4cba6e-efb4-03f1-302e-d5718c4eca20/9c4cba6f-8ea1-0202-ec4e-528291bf932e/AllDatarowsSlice/45e6dab8-c55e-4761-b47f-5a1dd4204f4d?view=carousel
https://app.labelbox.com/models/9c4cba6e-efb4-03f1-302e-d5718c4eca20/9c4cba6f-8ea1-0202-ec4e-528291bf932e/AllDatarowsSlice/746b2c7c-25e5-449e-8cce-50e4fcbb9ddd?view=carousel
https://app.labelbox.com/models/9c4cba6e-efb4-03f1-302e-d5718c4eca20/9c4cba6f-8ea1-0202-ec4e-528291bf932e/AllDatarowsSlice/7c2bd1f4-7f31-48b4-bc0a-b99ed7533646?view=carousel
